In [2]:
import requests
from bs4 import BeautifulSoup
import re
import xlwt

In [13]:
def get_HTML_text(url, stu_info):
    try:
        html = requests.get(url, params=stu_info, timeout = 300)
        html.raise_for_status()
        return html.text 
    except:
        return ''
    
def store_info(html_text, info_list):
    try:
        soup = BeautifulSoup(html_text, 'html.parser')
        raw_data_ls = soup.find_all('td', {"width": "20%", "align": "left"})
        pattern = re.compile(r'([\u4e00-\u9fa5]+):[\[]?([\d]*)[\]]?(.*)')
        matches = []
        for i in range(2, 5):
            matches.append(pattern.match(raw_data_ls[i].string))    
        info_list.append([matches[0].group(2), matches[1].group(2), matches[2].group(3)])
        
    except:
        pass

def write_to_Excel(info_list):
    f = xlwt.Workbook(encoding='utf-8')
    sheet1 = f.add_sheet('学生', cell_overwrite_ok=True)
    row0 = ["学号", "一卡通号", "姓名"]
    for i in range(0, len(row0)):
        sheet1.write(0,i,row0[i])
    
    for i in range(len(info_list)):
        for j in range(3):
            sheet1.write(i + 1, j, info_list[i][j])
    
    f.save('student_infomation.xls')
            

def print_student_info(info_list):
    tplt = "{:8}\t{:16}\t{:16}"
    print(tplt.format("学号", "一卡通号", "姓名"))
    for item in info_list:
        print( tplt.format(item[0], item[1], item[2]) )
        
def main():
    url = 'http://xk.urp.seu.edu.cn/jw_service/service/stuCurriculum.action'
    stu_info = {"returnStr": "", 
                "queryStudentId": '', 
                "queryAcademicYear": "18-19-3"}
    info_list = []
    init_id = 22015101
    for i in range(400):
        try:
            stu_info['queryStudentId'] = str(init_id + i)
            html_text = get_HTML_text(url, stu_info)
            store_info(html_text, info_list)
        except:
            continue
#     print_student_info(info_list)
    write_to_Excel(info_list)
    print('done')
    
main()

done
